In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.append('./src')

In [ ]:
df = pd.read_csv('ETTh1.csv', index_col=0)

In [ ]:
df.head()

In [ ]:
LOOKBACK = 336      # L: Lookback window
HORIZON = 96        # T: Predict next steps
PATCH_DIM = 16      # P: Patch size
STRIDE = 8          # S: Distance between start of subsequent patches
EMBED_DIM = 16      # D: Embedding dimensionality

# As suggested by the paper
NUM_PATCHES = (LOOKBACK - PATCH_DIM) // STRIDE + 2

In [ ]:
from src.dataset import ETTh1

train_dataset = ETTh1(file_path='./ETTh1.csv', split='train', lookback=LOOKBACK, horizon=HORIZON)
test_dataset = ETTh1(file_path='./ETTh1.csv', split='test', lookback=LOOKBACK, horizon=HORIZON)
val_dataset = ETTh1(file_path='./ETTh1.csv', split='val', lookback=LOOKBACK, horizon=HORIZON)

In [ ]:
sample = train_dataset[0]

sample['input'].shape, sample['target'].shape, sample['mean'], sample['std']

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
import torch

torch.cuda.is_available()

In [ ]:
from src.patch_tst import PatchTST

model = PatchTST(
    num_patches=NUM_PATCHES, 
    patch_dim=PATCH_DIM, 
    embed_dim=EMBED_DIM, 
    horizon=HORIZON,
    stride=STRIDE)

In [ ]:
from torch.optim import Adam

optimizer = Adam(params=model.parameters(), lr=1e-3)

In [ ]:
from tqdm import tqdm

EPOCHS = 10
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

model.to(DEVICE)
model.train()

for epoch in range(EPOCHS):
    # Training
    model.train()
    train_loss = 0
    train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{EPOCHS} [Train]')
    
    for batch in train_pbar:
        input = batch['input'].to(DEVICE)
        target = batch['target'].to(DEVICE)

        pred = model(input)
        loss = torch.nn.functional.mse_loss(pred, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        train_pbar.set_postfix({'loss': f'{loss.item():.4f}'})

    # Evaluation
    model.eval()
    test_loss = 0
    test_pbar = tqdm(test_loader, desc=f'Epoch {epoch+1}/{EPOCHS} [Test]')
    
    with torch.no_grad():
        for batch in test_pbar:
            input = batch['input'].to(DEVICE)
            target = batch['target'].to(DEVICE)

            pred = model(input)
            loss = torch.nn.functional.mse_loss(pred, target)
            test_loss += loss.item()
            test_pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    avg_train_loss = train_loss / len(train_loader)
    avg_test_loss = test_loss / len(test_loader)
    print(f'Epoch {epoch+1}/{EPOCHS} - Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}\n')

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
val_loss = 0
for batch in val_loader:
    input = batch['input'].to(DEVICE)
    target = batch['target'].to(DEVICE)
    mean = batch['mean'].to(DEVICE)
    std = batch['std'].to(DEVICE)

    pred = model(input)
    
    # Add dimension for broadcasting: (batch, features) -> (batch, features, 1)
    pred = pred * std.unsqueeze(-1) + mean.unsqueeze(-1)
    target = target * std.unsqueeze(-1) + mean.unsqueeze(-1)
    loss = torch.nn.functional.mse_loss(pred, target)
    val_loss += loss.item()
    print(f'Validation Loss: {loss.item():.4f}')

avg_val_loss = val_loss / len(val_loader)
print(f'Average Validation Loss: {avg_val_loss:.4f}')


In [ ]:
idx = 0
feature_idx = 0

input = val_dataset[idx]['input'].unsqueeze(0).to(DEVICE)
target = val_dataset[idx]['target'].unsqueeze(0).to(DEVICE)
mean = val_dataset[idx]['mean'].to(DEVICE).unsqueeze(-1)
std = val_dataset[idx]['std'].to(DEVICE).unsqueeze(-1)

pred = model(input)

# Denormalize
pred = pred * std + mean
target = target * std + mean

# Convert to numpy and select single feature
pred = pred.cpu().squeeze(0).numpy()  # Shape: (features, horizon)
target = target.cpu().squeeze(0).numpy()  # Shape: (features, horizon)

# Plot single feature
plt.figure(figsize=(12, 6))
plt.plot(pred[feature_idx], label='Predicted', marker='o')
plt.plot(target[feature_idx], label='Actual', marker='x')
plt.title(f'Predicted vs Actual Values - Feature {feature_idx}')
plt.xlabel('Time Steps')
plt.ylabel('Value')
plt.legend()
plt.grid()
plt.show()


NameError: name 'val_dataset' is not defined

In [ ]:
plt.subplots(4, 2, figsize=(15, 20))

features = ['OT', 'HUFL', 'HULL', 'MUFL', 'MULL', 'LUFL', 'LULL', 'OTHR']

for feature_idx in range(pred.shape[0]):
    plt.subplot(4, 2, feature_idx + 1)
    plt.plot(pred[feature_idx], label='Predicted', marker='o')
    plt.plot(target[feature_idx], label='Actual', marker='x')
    plt.title(f'{features[feature_idx]} - Predicted vs Actual')')
    plt.xlabel('Time Steps')
    plt.ylabel('Value')
    plt.legend()
    plt.grid()
plt.tight_layout()
plt.sav
plt.show()